https://zhuanlan.zhihu.com/p/34741174

https://blog.csdn.net/i_dovelemon/article/details/79598921

https://zhuanlan.zhihu.com/p/58641686

In [51]:
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mp

%matplotlib inline

In [ ]:
def RadicalInverse_VdC(bits):
    bits = (bits << 16) | (bits >> 16)
    bits = ((bits & 0x00ff00ff) << 8) | ((bits & 0xff00ff00) >> 8)
    bits = ((bits & 0x0f0f0f0f) << 4) | ((bits & 0xf0f0f0f0) >> 4)
    bits = ((bits & 0x33333333) << 2) | ((bits & 0xcccccccc) >> 2)
    bits = ((bits & 0x55555555) << 1) | ((bits & 0xaaaaaaaa) >> 1)
    return float(bits) * 2.3283064365386963e-10

def Hammersley(i, N):
    return float(i) / N, RadicalInverse_VdC(i)

def ImportanceSampleGGX(u, v, roughness):
    a = roughness * roughness
    phi = u * 2.0 * np.pi
    cosTheta = math.sqrt((1.0 - v) / (1.0 + (a * a - 1.0) * v))
    sinTheta = math.sqrt(1.0 - cosTheta * cosTheta)
    x = math.cos(phi) * sinTheta
    y = math.sin(phi) * sinTheta
    z = cosTheta
    return np.array([x, y, z])

def Vis_SmithJointApprox(roughness, nv, nl):
    a = roughness * roughness
    Vis_SmithV = nl * (nv * (1 - a) + a)
    Vis_SmithL = nv * (nl * (1 - a) + a)
    return 0.5 / (Vis_SmithV + Vis_SmithL + 1e-5)

def Normalize(x):
    return x / np.linalg.norm(x)

def GetCoordSpaceBasis(forward):
    up = np.array([0, 0, 1]) if abs(forward[2]) < 0.999 else np.array([1, 0, 0])
    right = Normalize(np.cross(up, forward))
    up = Normalize(np.cross(forward, right))
    return np.array([right, up, forward])
    
def TangentToWorld(v, tangentZ):
    return np.matmul(v, GetCoordSpaceBasis(tangentZ))
   
def IntegrateBRDF(NoV, roughness):
    SAMPLE_COUNT = 128
    N = np.array([0, 0, 1])
    V = np.array([np.sqrt(1.0 - NoV * NoV), 0.0, NoV])
    scale = 0
    bias = 0
    for i in range(SAMPLE_COUNT):
        x, y = Hammersley(i, SAMPLE_COUNT)
        H = TangentToWorld(ImportanceSampleGGX(x, y, roughness), N)
        L = 2 * np.dot(V, H) * H - V

        NoL = max(L[2], 0.0)
        NoH = max(H[2], 0.0)
        VoH = max(np.dot(V, H), 0.0)

        if (NoL > 0):
            Vis = Vis_SmithJointApprox(roughness, NoV, NoL) * 4.0 * NoL * VoH / NoH
            Fc = pow(1.0 - VoH, 5)
            scale += (1.0 - Fc) * Vis
            bias += Fc * Vis
            
    scale /= float(SAMPLE_COUNT)
    bias /= float(SAMPLE_COUNT)
    return scale, bias

size = 64
pixels = np.zeros((size, size, 3))
for i in range(size):
    for j in range(size):
        u = i / float(size)
        v = j / float(size)
        scale, bias = IntegrateBRDF(u, v)
        pixels[i][j][0] = scale
        pixels[i][j][1] = bias 

plt.title("LUT" + str(pixels.shape))
plt.imshow(pixels)
plt.show()
mp.imsave('images/BRDF_LUT.png', pixels)